## Libraries

In [15]:
import os
import yaml
import cv2
import numpy as np
import math
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon, shape
from shapely.affinity import scale
from rasterio.features import shapes
from rasterio import Affine
from shapely.wkt import loads
from shapely import hausdorff_distance
from shapely.validation import make_valid

# from shapely.geometry.base import geom_factory
# from shapely.geos import lgeos
# os.environ['USE_PYGEOS'] = '0'      # to avoid a bug in geopandas
# import geopandas as gpd

# Local Module Imports
import sys
sys.path.insert(0, "../src")
from utils import wkt2masc, load_images_from_folder, frame_to_label
from log_setup import logger
from eval import hausdorff_dist_wkt, iou_wkt
from visualization import create_boxplot

## Data Sources

In [3]:
# Base directory
current_dir = os.getcwd()
BASE_DIR = os.path.dirname(current_dir)

# Output directory
output_dir = "../outputs"

# Pyspatiotemporalgeom, Shape Based, CVAE output directories
pstg_dir = os.path.join(output_dir, "pstg")
shpe_dir = os.path.join(output_dir, "shape")
cvae_dir = os.path.join(output_dir, "CVAE")

# Load config file
config_file = os.path.join(BASE_DIR, "config.yml")
with open(config_file, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [6]:
MODE = "intepol" # "intepol" or "extrapol"
DATA = "full"    # "full" or "sampled"

### PNG to WKT conversion

In [ ]:
# List of generated frames paths
cvae_dir = os.path.join(cvae_dir, MODE, DATA)

## [BurnedAreaUAV](https://zenodo.org/records/7944963) Test Set Evalution

In [ ]:
logger.info("Reading WKTs files...")
with open(os.path.join(BASE_DIR, config["data"]["test_wkt"]), "r") as f:
    test_wkt = f.read().splitlines()

with open(os.path.join(BASE_DIR, cvae_dir, MODE, DATA, "WKT"), "r") as f:
    cvae_wkt = f.read().splitlines()

with open(os.path.join(BASE_DIR, shpe_dir, MODE, DATA, "WKT"), "r") as f:
    shp_wkt = f.read().splitlines()

with open(os.path.join(BASE_DIR, pstg_dir, MODE, DATA, "WKT"), "r") as f:
    pstg_wkt = f.read().splitlines()


# test set frame indexes
idx = np.linspace(20250, 22450, 23).astype(int)

logger.info("Selecting test set corresponding lines only...")
shp_wkt = [shp_wkt[i] for i in idx]
cvae_wkt = [cvae_wkt[i] for i in idx]
pstg_wkt = [pstg_wkt[i] for i in idx]

logger.info(f"Shape based {MODE}ation: {len(shp_wkt):>5} frames")
logger.info(f"Conditional VAE {MODE}ation: {len(cvae_wkt):>5} frames")
logger.info(f"McKenney {MODE}ation: {len(pstg_wkt):>5} frames")
logger.info("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
logger.info(f"BurnedAreaUAV Test Set: {len(test_wkt):>5} frames")

### Hausdorf Distance

In [ ]:
cvae_mean_hd, cvae_hd_list = hausdorff_dist_wkt(test_wkt, cvae_wkt)
shp_mean_hd, shp_hd_list = hausdorff_dist_wkt(test_wkt, shp_wkt)
pstg_mean_hd, pstg_hd_list = hausdorff_dist_wkt(test_wkt, pstg_wkt)

logger.info(" Hausdorff Distance - Burned Area UAV Test Set")
logger.info(
    f"CVAE - {MODE}ation:\n"
    f"\t Mean: {cvae_mean_hd:.4f}\n"
    f"\t Std: {np.std(cvae_hd_list):.4f}\n"
    f"\t Min: {np.min(cvae_hd_list):.4f}\n"
    f"\t Max: {np.max(cvae_hd_list):.4f}"
)
logger.info(
    f"Shape based - {MODE}ation:\n"
    f"\t Mean: {shp_mean_hd:.4f}\n"
    f"\t Std: {np.std(shp_hd_list):.4f}\n"
    f"\t Min: {np.min(shp_hd_list):.4f}\n"
    f"\t Max: {np.max(shp_hd_list):.4f}"
)
logger.info(
    f"McKenney - {MODE}ation:\n"
    f"\t Mean: {pstg_mean_hd:.4f}\n"
    f"\t Std: {np.std(pstg_hd_list):.4f}\n"
    f"\t Min: {np.min(pstg_hd_list):.4f}\n"
    f"\t Max: {np.max(pstg_hd_list):.4f}"
)

In [ ]:
# Store data in a dictionary
data = {
    "Shape Based": {"full": hd_list_shp_ts, "sampled": hd_list_shp_pr},
    "CVAE": {"full": hd_list_cvae_ts, "sampled": hd_list_cvae_pr},
    "McKenney": {"full": hd_list_pstg_ts, "sampled": hd_list_pstg_pr},
}

# Positions for bars
positions = {"full": [1, 1.5, 2], "sampled": [2.75, 3.25, 3.75]}

# Colors for boxes
colors = ["lightgreen", "lightblue", "lightcoral"]

# Create boxplot
create_boxplot(data, positions, colors)

### IoU 

In [ ]:
cvae_mean_iou, cvae_iou_list = iou_wkt(test_wkt, cvae_wkt)
shp_mean_iou, shp_iou_list = iou_wkt(test_wkt, shp_wkt)
pstg_mean_iou, pstg_iou_list = iou_wkt(test_wkt, pstg_wkt)

logger.info(" IoU Scores - Burned Area UAV Test Set")
logger.info(
    f"CVAE - {MODE}ation:\n"
    f"\t Mean: {cvae_mean_iou:.4f}\n"
    f"\t Std: {np.std(cvae_iou_list):.4f}\n"
    f"\t Min: {np.min(cvae_iou_list):.4f}\n"
    f"\t Max: {np.max(cvae_iou_list):.4f}"
)
logger.info(
    f"Shape based - {MODE}ation:\n"
    f"\t Mean: {shp_mean_iou:.4f}\n"
    f"\t Std: {np.std(shp_iou_list):.4f}\n"
    f"\t Min: {np.min(shp_iou_list):.4f}\n"
    f"\t Max: {np.max(shp_iou_list):.4f}"
)
logger.info(
    f"McKenney - {MODE}ation:\n"
    f"\t Mean: {pstg_mean_iou:.4f}\n"
    f"\t Std: {np.std(pstg_iou_list):.4f}\n"
    f"\t Min: {np.min(pstg_iou_list):.4f}\n"
    f"\t Max: {np.max(pstg_iou_list):.4f}"
)

## U-Net Generated Polygons Comparison

In [16]:
# Read data/segs_unet_base.wkt
with open(os.path.join(BASE_DIR, "data/segs_unet_base.wkt"), "r") as f:
    unet_wkt = f.read().splitlines()

In [ ]:
cvae_mean_iou, cvae_iou_list = iou_wkt(unet_wkt, cvae_wkt)
shp_mean_iou, shp_iou_list = iou_wkt(unet_wkt, shp_wkt)
pstg_mean_iou, pstg_iou_list = iou_wkt(unet_wkt, pstg_wkt)

logger.info(" IoU Scores - UNet Test Set")
logger.info(
    f"CVAE - {MODE}ation:\n"
    f"\t Mean: {cvae_mean_iou:.4f}\n"
    f"\t Std: {np.std(cvae_iou_list):.4f}\n"
    f"\t Min: {np.min(cvae_iou_list):.4f}\n"
    f"\t Max: {np.max(cvae_iou_list):.4f}"
)
logger.info(
    f"Shape based - {MODE}ation:\n"
    f"\t Mean: {shp_mean_iou:.4f}\n"
    f"\t Std: {np.std(shp_iou_list):.4f}\n"
    f"\t Min: {np.min(shp_iou_list):.4f}\n"
    f"\t Max: {np.max(shp_iou_list):.4f}"
)
logger.info(
    f"McKenney - {MODE}ation:\n"
    f"\t Mean: {pstg_mean_iou:.4f}\n"
    f"\t Std: {np.std(pstg_iou_list):.4f}\n"
    f"\t Min: {np.min(pstg_iou_list):.4f}\n"
    f"\t Max: {np.max(pstg_iou_list):.4f}"
)